# Creating a loader with Kosh

In this example we will create a custom loader for some ASCII representation

The structure is
* Headers at the begining of the file starting with #
* *#* varname vs axis
 * indicates new variable with name `varname`
* var_value axis_value
 * repeated n times
* end
 * marks end of the current variable

We will assume the user already has functions to read the data in. These can be found in the [some_user_io_functions.py](some_user_io_functions.py) file.

The function to read data in is called `load_variable_from_file`, the one to list the features in the file is called `get_variable_names`.

In [1]:
import os
import kosh

# Make sure local file is new sql file
kosh_example_sql_file = "my_store.sql"
    
# Create db on file
store = kosh.connect(kosh_example_sql_file, delete_all_contents=True)

In [2]:
# Add dataset to the store
sample = store.create(name="example", metadata={'project':"example"})

In [3]:
# Associate file with datasets
sample.associate("example.ultra", mime_type="custom")

'f17daf05c9204a868b642e7b178a3c53'

Let's create our `CustomLoader` inheriting from `kosh.KoshLoader`

For this we will need to:

* register the types we can read at the class level (not in `__init__`) and the format each type can be exported as.
 * types : { "custom" : ["numpy",] }
 * **IMPORTANT** the keys in this dictionary is what Kosh uses to tie the loader to a **mime_type**
* implement the `extract` function to read data in
 * desired feature is in `self.feature`
 * potential keywords are in: `self._user_passed_parameters[1]`
 * kosh object describing the source is in `self.obj` (can query its attributes if desired)
 * source uri is at: `self.obj.uri`
 * The function to read a variable from a file is: `load_variable_from_file`
* implement the `list_features(self)` function, using the `get_variable_names` helper function.
* optionally implement the `describe_feature(self, feature)`


In [4]:
import sys, os
sys.path.append(".")
from some_user_io_functions import load_variable_from_file, get_variable_names

Let's query the function documentation

In [5]:
load_variable_from_file?

Signature: load_variable_from_file(filepath, variable_names)
Docstring:
Load the variable 'variable_name' for a file at filepath
:param filepath: path to the file to read
:type filepath: str
:param variable_names: Name of the variable(s) to read in file
:type variable_names: str or list
:return: A numpy array containing the variable(s) values
:rtype: numpy.ndarray
File:      ~/git/kosh/examples/some_user_io_functions.py
Type:      function


In [6]:
from kosh import KoshLoader
import numpy

class CustomLoader(KoshLoader):
    types ={"custom": ["numpy", ]}  # keys ties things back to mime_type in associate function
    def extract(self, *args, **kargs):
        return load_variable_from_file(self.obj.uri, self.feature)
        
    def list_features(self):
        return get_variable_names(self.obj.uri)
    
    def describe_feature(self, feature):
        var = load_variable_from_file(self.obj.uri, feature)
        info = {"name": feature, "size": var.shape}
        return info


At this point we need to register/[add](https://lc.llnl.gov/kosh/autoapi/kosh/core/index.html#kosh.core.KoshStoreClass.add_loader) our loader with the store (let's save it in the store as well).

In [7]:
store.add_loader(CustomLoader)

We can now [query](https://lc.llnl.gov/kosh/autoapi/kosh/core/index.html#kosh.core.KoshDataset.list_features) our dataset, as explained in the [previous](04_Associating_and_Reading_Data.ipynb) notebook.

In [8]:
print(sample.list_features())

['time', 'energy', 'var2']


In [9]:
sample.describe_feature("energy")

{'name': 'energy', 'size': (8,)}

Or extract its features

In [10]:
print(sample.get("energy"))

[0.6 0.7 0.8 0.6 0.5 0.2 0.1 0.6]
